<a href="https://colab.research.google.com/github/UjalaJha/NasaSpaceAppChallenge/blob/master/Fire_detection_classification/Fire_severity_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthpy seaborn geopandas flask-restful flask-ngrok

     |████████████████████████████████| 1.4MB 4.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 972kB 14.9MB/s 
     |████████████████████████████████| 18.1MB 222kB/s 
     |████████████████████████████████| 14.8MB 250kB/s 
     |████████████████████████████████| 10.9MB 18.6MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for earthpy: filename=earthpy-0.9.2-cp36-none-any.whl size=1365088 sha256=ec75c914e9d081ac28a238c2c2d1daed44f08e01163bb7245455078fb141352d
  Stored in directory: /root/.cache/pip/wheels/27/f9/a5/026bb95ef7b12f9343b842da6897b4001c5b902550095cbbbe
Successfully built earthpy


In [ ]:
from glob import glob
import os
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib import colors
import matplotlib as mpl
import seaborn as sns

import rasterio as rio
from rasterio.plot import plotting_extent, show
from rasterio.plot import reshape_as_raster, reshape_as_image

import geopandas as gpd
from shapely.geometry import mapping, box

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import flask
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask_restful import Resource, Api
import json

sns.set_style('white')
sns.set(font_scale=1.5)

# data1 = et.data.get_data('cold-springs-fire')
# data2 = et.data.get_data('cs-test-landsat')
# os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
def show_band(img, title='', size=(6,6), cmap='viridis'):
  print(title+' Shape: ', img.shape)
  fig = plt.figure(figsize=size)
  plt.imshow(img.reshape(img.shape[0], img.shape[1]), cmap= cmap)
  plt.axis('off')
  plt.colorbar()
  plt.title(title)
  plt.show()

def nbr(band1, band2):
    """
    This function takes an input the arrays of the bands from the read_band_image
    function and returns the Normalized Burn ratio (NBR)
    input:  band1   array (n x m)      array of first band image e.g B8A
            band2   array (n x m)      array of second band image e.g. B12
    output: nbr     array (n x m)      normalized burn ratio
    """
    nbr = (band1 - band2) / (band1 + band2)
    return nbr

def dnbr(nbr1,nbr2):
    """
    This function takes as input the pre- and post-fire NBR and returns the dNBR
    input:  nbr1     array (n x m)       pre-fire NBR
            nbr2     array (n x m)       post-fire NBR
    output: dnbr     array (n x m)       dNBR
    """
    dnbr = nbr1 - nbr2
    return dnbr

In [ ]:
def classify():
  # Identify band-paths
  sentinel_pre_fire_bands = glob("sentinel-pre-fire/*.jp2")
  sentinel_post_fire_bands = glob("sentinel-post-fire/*.jp2")

  sentinel_pre_fire_bands.sort()
  sentinel_post_fire_bands.sort()

  print(sentinel_pre_fire_bands)
  print(sentinel_post_fire_bands)

  # Read the pre-fire band images
  with rio.open(sentinel_pre_fire_bands[3]) as f1:
    SWIR = reshape_as_image(f1.read())

  with rio.open(sentinel_pre_fire_bands[4]) as f1:
    NIR = reshape_as_image(f1.read())

  # Calculation of pre-fire NBR
  pre_fire_nbr = nbr(NIR.astype(int), SWIR.astype(int))

  # Read the post-fire band images
  with rio.open(sentinel_post_fire_bands[3]) as f1:
      SWIR = reshape_as_image(f1.read())

  with rio.open(sentinel_post_fire_bands[4]) as f1:
      NIR = reshape_as_image(f1.read())

  # Calculation of post-fire NBR
  post_fire_nbr = nbr(NIR.astype(int), SWIR.astype(int))

  # Calculation of dNBR
  DNBR = dnbr(pre_fire_nbr, post_fire_nbr)

  dnbr_sentinel_class = get_sentinel_class(DNBR)
  
  affected_areas = get_area(dnbr_sentinel_class)

  data = [dnbr_sentinel_class, affected_areas]
  return json.dumps(data, cls=NumpyEncoder)

  # return plot_colormap(dnbr_sentinel_class)

In [ ]:
def get_sentinel_class(DNBR):
  dnbr_class_bins = [-np.inf, -0.1, 0.1, 0.27, 0.66, np.inf]
  dnbr_sentinel_class = np.digitize(DNBR, dnbr_class_bins)
  return dnbr_sentinel_class

In [ ]:
def plot_colormap(dnbr_sentinel_class):
  dnbr_cat_names = ["Enhanced Regrowth",
                    "Unburned",
                    "Low Severity",
                    "Moderate Severity",
                    "High Severity"]

  nbr_colors = ["g", "yellowgreen", "peachpuff", "coral", "maroon"]

  nbr_cmap = ListedColormap(nbr_colors)

  # Plot the data with a custom legend
  fig, ax = plt.subplots(figsize=(10, 8))
  im = ax.imshow(dnbr_sentinel_class.reshape(dnbr_sentinel_class.shape[:2]), cmap=nbr_cmap)

  ax.set_title("Sentinel dNBR",
              fontsize=16)

  cbar = ep.colorbar(im)

  cbar.set_ticks(np.unique(dnbr_sentinel_class))
  cbar.set_ticklabels(dnbr_cat_names)

  # Turn off ticks
  ax.set_axis_off()
  return plt.show()

In [ ]:
# # To calculate area, multiply the number of pixels in each bin by image resolution

def get_area(dnbr_sentinel_class):
  sentinel_pixel_size = 20 * 20
  enhanced_regrowth = (dnbr_sentinel_class[dnbr_sentinel_class == 1]).size
  unburned = (dnbr_sentinel_class[dnbr_sentinel_class == 2]).size
  low = (dnbr_sentinel_class[dnbr_sentinel_class == 3]).size
  moderate = (dnbr_sentinel_class[dnbr_sentinel_class == 4]).size
  high = (dnbr_sentinel_class[dnbr_sentinel_class == 5]).size


  enhanced_regrowth = np.multiply(enhanced_regrowth, sentinel_pixel_size)/1000000
  unburned = np.multiply(unburned, sentinel_pixel_size)/1000000
  low = np.multiply(low, sentinel_pixel_size)/1000000
  moderate = np.multiply(moderate, sentinel_pixel_size)/1000000
  high = np.multiply(high, sentinel_pixel_size)/1000000

  affected_areas = {
      'enhanced_regrowth': enhanced_regrowth,
      'unburned': unburned,
      'low': low,
      'moderate': moderate,
      'high': high,
  }
  return affected_areas

In [ ]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)   #starts ngrok when the app is run

class Classify(Resource):
    def get(self, keywords):
      # Change directory into the specified case study
      os.chdir("/content/drive/My Drive/spaceapps_data/{}".format(keywords))
      # Call the classify method here and return the output
      return classify()

api.add_resource(Classify, '/classify/<string:keywords>')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://07636859cd67.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
